In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Define the CNN architecture
class SimpleConvNet(nn.Module):
    def __init__(self):
        super(SimpleConvNet, self).__init__()
        # First convolutional layer
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(2, 2)
        
        # Second convolutional layer
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(2, 2)
        
        # Fully connected layers
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.relu3 = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        # First conv block
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        
        # Second conv block
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        
        # Flatten the output
        x = x.view(-1, 64 * 7 * 7)
        
        # Fully connected layers
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x

# Training function
def train_model(model, train_loader, criterion, optimizer, device, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        
        for i, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)
            
            # Zero the gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # Backward pass and optimize
            loss.backward()
            optimizer.step()
            
            # Calculate accuracy
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            running_loss += loss.item()
            
            if (i + 1) % 100 == 0:
                print(f'Epoch [{epoch + 1}/{epochs}], Step [{i + 1}/{len(train_loader)}], '
                      f'Loss: {running_loss / 100:.4f}, '
                      f'Accuracy: {100 * correct / total:.2f}%')
                running_loss = 0.0

# Main training script
def main():
    # Set device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Define transforms
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
    
    # Load Fashion MNIST dataset
    train_dataset = torchvision.datasets.FashionMNIST(
        root='./data',
        train=True,
        transform=transform,
        download=True
    )
    
    train_loader = DataLoader(
        dataset=train_dataset,
        batch_size=64,
        shuffle=True
    )
    
    # Initialize the model
    model = SimpleConvNet().to(device)
    
    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # Train the model
    train_model(model, train_loader, criterion, optimizer, device)

if __name__ == '__main__':
    main()

Epoch [1/10], Step [100/938], Loss: 1.0785, Accuracy: 61.12%
Epoch [1/10], Step [200/938], Loss: 0.6518, Accuracy: 68.77%
Epoch [1/10], Step [300/938], Loss: 0.5852, Accuracy: 72.09%
Epoch [1/10], Step [400/938], Loss: 0.5235, Accuracy: 74.40%
Epoch [1/10], Step [500/938], Loss: 0.4867, Accuracy: 75.99%
Epoch [1/10], Step [600/938], Loss: 0.4724, Accuracy: 77.24%
Epoch [1/10], Step [700/938], Loss: 0.4455, Accuracy: 78.21%
Epoch [1/10], Step [800/938], Loss: 0.4530, Accuracy: 78.92%
Epoch [1/10], Step [900/938], Loss: 0.4126, Accuracy: 79.69%
Epoch [2/10], Step [100/938], Loss: 0.4006, Accuracy: 85.64%
Epoch [2/10], Step [200/938], Loss: 0.3913, Accuracy: 85.69%
Epoch [2/10], Step [300/938], Loss: 0.3616, Accuracy: 86.04%
Epoch [2/10], Step [400/938], Loss: 0.3664, Accuracy: 86.36%
Epoch [2/10], Step [500/938], Loss: 0.3623, Accuracy: 86.51%
Epoch [2/10], Step [600/938], Loss: 0.3577, Accuracy: 86.60%
Epoch [2/10], Step [700/938], Loss: 0.3495, Accuracy: 86.75%
Epoch [2/10], Step [800/